<a href="https://colab.research.google.com/github/yeonghun00/stock_public/blob/main/crawler/KOTC_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import ast

class Stock:
  def __init__(self, code):
    self.code = code
    self.headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}

  def get_price(self):
    url = 'http://kotc.kisline.com/highlight/mainHighlight.nice?paper_stock=' + self.code + '&nav=1'
    result = requests.get(url)
    bs_obj = BeautifulSoup(result.content, "html.parser")
    b = bs_obj.get_text()
    for i in ['\n', '\t', "\\", ' ']:
      b = b.replace(i,'')

    data = np.array(ast.literal_eval(b)[1:])

    dic = {'Date':list(map(lambda x: datetime.datetime.strptime(str(x), '%Y%m%d'), data[:,0])), \
      'Open':np.array(data[:,1], int), 'High':np.array(data[:,2], int), 'Low':np.array(data[:,3], int),\
       'Close':np.array(data[:,4], int), 'Volume':np.array(data[:,5], int)}

    df = pd.DataFrame(data=dic)
    df = df.set_index('Date')
    return df
    
  def get_fundamental(self):
    url = 'https://finance.naver.com/item/main.nhn?code=' + self.code
    result = requests.get(url, headers = self.headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")

    ths = bs_obj.find_all("th", {'scope':'col'})
    ths = [th.get_text() for th in ths][10:-22]
    dates = list(map(lambda x: x.translate(str.maketrans('','','\n\t, ')),ths))
    dates = list(map(lambda x: x + '(Y)', dates[:4])) + list(map(lambda x: x + '(M)', dates[4:]))

    tr = bs_obj.find_all("tbody")
    tds = tr[2].find_all('td')
    tds = [td.get_text() for td in tds]
    elements = list(map(lambda x: x.translate(str.maketrans('','','\n\t, ')),tds))
    elements = list(map(lambda x: float(x) if x.replace('.','').isdigit() else np.nan, elements))

    temp_dict = {}
    cnt = 0
    index = ['sales', 'operating profit', 'net income', 'operating margin', 'net margin', 'roe', 'debt ratio', 'quick ratio', \
    'reserve ratio', 'eps', 'per', 'bps', 'pbr', 'dividend per share', 'dividend yield ratio', 'dividend payout ratio']

    for i in dates:
      temp_dict[i] = elements[cnt::10]
      cnt += 1

    df = pd.DataFrame.from_dict(temp_dict)
    df = df.set_index([pd.Index(index)])
    return df

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import ast


In [34]:
url = 'http://kotc.kisline.com/highlight/mainHighlight.nice?paper_stock=227680&nav=1'
result = requests.get(url)
bs_obj = BeautifulSoup(result.content, "html.parser")

summary = bs_obj.find_all('table', {'class':'list_b1', 'summary':'기업소개'})[0].find_all('li')
summary = ' '.join([x.text for x in summary])

In [35]:
summary

'뉴젠홀딩스(주)는 2002년 1월 소프트웨어 개발 및 공급 등을 목적으로 설립되어 2008년 7월 법인전환하였으며, 2017년 3월 현재의 상호로 변경하고 경영정보화솔루션 개발 및 공급업을 영위 중에 있음. 2013년 11월 스마트M그룹웨어 제품을 출시하고, 2014년 6월 ERP연동 계좌조회서비스를 출시하였으며, 9월에는 선택적 기업복지서비스인 웰마켓(복지몰)을 출시하였음. 2018년 3월 웰마켓 사업부문을 물적분할하여 웰마켓(주)로 신설하였으나, 2020년 3월 대표이사 등에게 매각하였음. 현재 동사는 ERP시스템 사업부문을 주력사업으로 영위하고 있으며, 바코드ERP, 프로젝트ERP, 모바일ERP, 오픈마켓ERP(샵엔ERP) 등의 다양한 솔루션을 제공하고 있음. 한편, 2013년 기업부설연구소를 설립하여 연구개발을 지속하고 있음. 2020년 결산 기준 매출액은 코로나19 여파로 정부의 비대면 서비스 바우처 지원사업 대상자로 선정되어 동사의 뉴젠 ERP 솔루션 수주가 호조를 보인 가운데 유지보수 수주도 증가하여 전년대비 122.38% 증가한 2,843백만원을 기록, 큰 폭의 매출 성장세를 시현하였음. 큰 폭의 매출 성장으로 판관비 부담이 완화되었으나 원가구조 저하로 전년대비 영업이익률이 크게 하락하였으며, 순이익은 적자전환하였음.(영업이익률 2.29%, 당기순손실 40백만원) (주)투비소프트 지분 취득으로 발생한 매도가능증권평가이익의 기타자본 반영으로 자기자본이 확충되었으나 차입금 증가 등으로 부채 규모가 확대되어 전년대비 제안정성 지표 저하, 자기자본비율 51.59%, 부채비율 93.83%, 차입금의존도 45.16%로 자산대비 차입금이 과중한 수준을 보임. 정부의 디지털뉴딜 정책사업과 함께 K비대면 바우처 플랫폼 내 높은 평점을 기록하며 제품경쟁력을 확보하고 있는 바, ERP 솔루션 수주가 지속되며 양호한 매출 성장세 이어갈 것으로 예상됨. 또한, 이에 따른 고정비 부담의 완화로 수익구조도 개선될 것으로 예상됨.'

In [40]:
annual = bs_obj.find_all('table', {'class':'list_b1', 'summary':'매출액'})[0]
annual

<table class="list_b1" style="float:left;width:50%;border-right:3px double #b7b7b7;" summary="매출액">
<caption>상세내용</caption>
<colgroup>
<col width="12%"/>
<col width="8%"/>
<col width="8%"/>
<col width="8%"/>
<col width="8%"/>
</colgroup>
<thead>
<tr>
<th class="first" scope="col"> </th>
<th colspan="4" scope="col">연간재무제표(Annual)</th>
</tr>
<tr>
<th class="first" scope="col">구분</th>
<th scope="col">2017.12</th>
<th scope="col">2018.12</th>
<th scope="col">2019.12</th>
<th scope="col">2020.12</th>
</tr>
</thead>
<tbody>
<tr>
<th scope="row">매출액(억원)</th>
<td class="rgt">5</td>
<td class="rgt">6</td>
<td class="rgt">13</td>
<td class="rgt">28</td>
</tr>
<tr class="bgc_gry">
<th scope="row">영업이익(억원)</th>
<td class="rgt">0</td>
<td class="rgt">1</td>
<td class="rgt">2</td>
<td class="rgt">1</td>
</tr>
<tr>
<th scope="row">영업이익율(%)</th>
<td class="rgt">8.22</td>
<td class="rgt">12.26</td>
<td class="rgt">16.85</td>
<td class="rgt">2.29</td>
</tr>
<tr class="bgc_gry">
<th scope="row">당기순이익(억원)

In [ ]:
dic = {}

In [43]:
annual.find_all('th')

[<th class="first" scope="col"> </th>,
 <th colspan="4" scope="col">연간재무제표(Annual)</th>,
 <th class="first" scope="col">구분</th>,
 <th scope="col">2017.12</th>,
 <th scope="col">2018.12</th>,
 <th scope="col">2019.12</th>,
 <th scope="col">2020.12</th>,
 <th scope="row">매출액(억원)</th>,
 <th scope="row">영업이익(억원)</th>,
 <th scope="row">영업이익율(%)</th>,
 <th scope="row">당기순이익(억원)</th>,
 <th scope="row">순이익율(%)</th>,
 <th scope="row">자산총계(억원)</th>,
 <th scope="row">부채총계(억원)</th>,
 <th scope="row">자본총계(억원)</th>,
 <th scope="row">유보율(%)</th>,
 <th scope="row">ROE</th>,
 <th scope="row">PER</th>,
 <th scope="row">PBR</th>,
 <th scope="row">PSR</th>,
 <th scope="row">EPS(원)</th>,
 <th scope="row">BPS(원)</th>,
 <th scope="row">SPS(원)</th>]

In [42]:
[x.text for x in annual.find_all('td')]

['5',
 '6',
 '13',
 '28',
 '0',
 '1',
 '2',
 '1',
 '8.22',
 '12.26',
 '16.85',
 '2.29',
 '0',
 '0',
 '1',
 '0',
 '3.4',
 '3.24',
 '9.45',
 '-1.4',
 '44',
 '53',
 '74',
 '166',
 '12',
 '21',
 '31',
 '80',
 '32',
 '32',
 '43',
 '86',
 '24.77',
 '25.53',
 '24.29',
 '144.51',
 '0.65',
 '0.61',
 '3.21',
 '-0.62',
 '530',
 '1,500',
 '0',
 '0',
 '4.77',
 '13.76',
 '4.81',
 '1.71',
 '0',
 '62.5',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '111',
 '109',
 '113',
 '230',
 '0',
 '24',
 '0',
 '0']